In [3]:
import pandas, os, math
pd = pandas

results_df = {}
results_df['CountryandRegionMap__c'] = pd.read_excel('TaxCalcFiles/CountryandRegionMap__c.xlsx',index_col=0)
results_df['CPA_Costing__c'] = pd.read_excel('TaxCalcFiles/CPA_Costing__c.xlsx',index_col=0)
results_df['Currency_Management2__c'] = pd.read_excel('TaxCalcFiles/Currency_Management2__c.xlsx',index_col=0)
results_df['Freight__c'] = pd.read_excel('TaxCalcFiles/Freight__c.xlsx',index_col=0)
results_df['Part__c'] = pd.read_excel('TaxCalcFiles/Part__c.xlsx',index_col=0)
results_df['Shipment_Order__c'] = pd.read_excel('TaxCalcFiles/Shipment_Order__c.xlsx',index_col=0)
results_df['Shipment_Order_Package__c'] = pd.read_excel('TaxCalcFiles/Shipment_Order_Package__c.xlsx',index_col=0)
results_df['Task'] = pd.read_excel('TaxCalcFiles/Task.xlsx',index_col=0)
results_df['Tax_Condition__c'] = pd.read_excel('TaxCalcFiles/Tax_Condition__c.xlsx',index_col=0)
results_df['Tax_Structure_Per_Country__c'] = pd.read_excel('TaxCalcFiles/Tax_Structure_Per_Country__c.xlsx',index_col=0)

data = results_df.copy()

In [100]:
def get_SOs(dataset, so_lookup_value):
    data = dataset.copy()
    data = data['Shipment_Order__c']
    so_data = data[data['Lookup Value'] == so_lookup_value]
    return so_data

def get_LIs(dataset, so_lookup_value):
    data = dataset.copy()
    data = data['Part__c']
    return data[data['Shipment_Order__r.Client_Reference__c'] == so_lookup_value]

def get_tax_structure(dataset, clearance_country):
    data = dataset.copy()
    tax_structures = data['Tax_Structure_Per_Country__c']
    tax_structures[tax_structures['Country__r.Name'] == clearance_country]
    return tax_structures[tax_structures['Country__r.Name'] == clearance_country]     

def get_tax_type(tax_structure_item):
    tax_type = tax_structure_item['Tax_Type__c'][0]
    tax_rate = tax_structure_item['Rate__c'][0]
    tax_amount = tax_structure_item['Amount__c'][0]
    tax_part_specific = tax_structure_item['Part_Specific__c'][0]
    if tax_type == 'VAT' and tax_part_specific == True:
        return 'VAT'
    elif tax_type == 'Duties' and tax_part_specific == True:
        return 'Duties Part Specific'
    elif tax_type != 'Duties' and tax_part_specific == True:
        return 'Other Part Specific'
    elif pd.isna(tax_rate):
        return 'Amount'
    else:
        return 'Rate Non Part Specific'   

def get_tax_conditions(dataset, tax_structure_item):
    data = dataset.copy()
    tax_calc_id = tax_structure_item['Id'][0]
    tax_conditions = data['Tax_Condition__c']
    return tax_conditions[tax_conditions['Tax_Structure_Per_Country__c'] == tax_calc_id]     

def get_tax_amount(tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    so_tax_amount = tax_structure_item['Amount__c'][0]
    so_has_line_items = so_tax_item['of_Line_Items__c'][0] > 0
    number_of_line_items = so_tax_item['of_Line_Items__c'][0]
    shipment_value = so_tax_item['Shipment_Value_USD__c'][0]
    
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c'][0]
        li_proportion = 1 / number_of_line_items
        #print(li_proportion)
        tax_amount = so_tax_amount * li_proportion
        #print(tax_amount)
        return tax_amount
    else:
        return so_tax_amount

In [141]:
def get_tax_rate(tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    tax_structure_item_rate = tax_structure_item['Rate__c'][0]/100
    tax_structure_item_default_rate = tax_structure_item['Default_Duty_Rate__c'][0]/100
    so_has_line_items = so_tax_item['of_Line_Items__c'][0] > 0
    
    if tax_type == 'VAT':
        if so_has_line_items:
            if pd.isna(li_tax_item['Specific_VAT_Rate__c'][0]):
                tax_item_rate = tax_structure_item_rate
            else:
                tax_item_rate = li_tax_item['Specific_VAT_Rate__c'][0]/100
        else:
            tax_item_rate = tax_structure_item_rate
    
    elif tax_type == 'Duties Part Specific':
        if so_has_line_items:
            if pd.isna(li_tax_item['Rate__c'][0]):
                tax_item_rate = tax_structure_item_default_rate
            else:
                tax_item_rate = li_tax_item['Rate__c'][0]/100
        else:
            tax_item_rate = tax_structure_item_default_rate
    
    elif tax_type == 'Other Part Specific':
        if so_has_line_items:
            if pd.isna(li_tax_item['Part_Specific_Rate_1__c'][0]):
                tax_item_additional_rate_1 = 0
            else:
                tax_item_additional_rate_1 = li_tax_item['Part_Specific_Rate_1__c'][0]/100
            if pd.isna(li_tax_item['Part_Specific_Rate_2__c'][0]):
                tax_item_additional_rate_2 = 0
            else:
                tax_item_additional_rate_2 = li_tax_item['Part_Specific_Rate_2__c'][0]/100
            if pd.isna(li_tax_item['Part_Specific_Rate_3__c'][0]):
                tax_item_additional_rate_3 = 0
            else:
                tax_item_additional_rate_3 = li_tax_item['Part_Specific_Rate_3__c'][0]/100
            tax_structure_item_additional = int(tax_structure_item['Additional_Part_Specific_Tax__c'][0])
            tax_item_rate = eval(f'tax_item_additional_rate_{tax_structure_item_additional}')
        else:
            tax_item_rate = 0
    
    elif tax_type == 'Rate Non Part Specific':
        tax_item_rate = tax_structure_item_rate        
          
    return tax_item_rate

In [114]:
def get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    applied_to_string = tax_structure_item['Applied_to_Value__c'][0]
    order_number = tax_structure_item['Order_Number__c'][0]
    additional_percent = tax_structure_item['Additional_Percent__c'][0]
    additional_applies_to = tax_structure_item['Applies_to_Order__c'][0]
    so_has_line_items = so_tax_item['of_Line_Items__c'][0] > 0
    so_lookup_value = so_tax_item['Lookup Value'][0]
    
    if (tax_type == 'VAT' or tax_type == 'Duties Part Specific' or tax_type == 'Other Part Specific' or tax_type == 'Rate Non Part Specific'):
        if so_has_line_items:
            li_lookup_value = li_tax_item['Lookup Value'][0]
            if applied_to_string == 'CIF':
                taxable_value = get_cif_value(tax_structure_item, so_tax_item, li_tax_item)
            elif applied_to_string == 'FOB':
                taxable_value = get_fob_value(tax_structure_item, so_tax_item, li_tax_item)
            else:
                taxable_value = get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,applied_to_string,li_lookup_value)
            
        else:
            if applied_to_string == 'CIF':
                taxable_value = get_cif_value(tax_structure_item, so_tax_item)
            elif applied_to_string == 'FOB':
                taxable_value = get_fob_value(tax_structure_item, so_tax_item)
            else:
                taxable_value = get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,applied_to_string,so_lookup_value)
                #print(taxable_value)
        
        if not pd.isna(additional_percent):
            taxable_value = taxable_value * (1 + additional_percent/100)
        
        if not pd.isna(additional_applies_to):
            for additional_applies_to_item in additional_applies_to.split(','):
                if so_has_line_items:
                    taxable_value += get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,additional_applies_to_item,li_lookup_value)
                else:
                    taxable_value += get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,additional_applies_to_item,so_lookup_value)
                    
    return taxable_value


In [115]:
def get_cif_value(tax_structure_item,so_tax_item,li_tax_item={}):
    cif_absolute_value_adjustment = tax_structure_item['Country__r.CIF_Absolute_value_adjustment__c'][0]
    cif_adjustment_factor = tax_structure_item['Country__r.CIF_Adjustment_Factor__c'][0]/100
    
    cpa_cif = so_tax_item['CPA_v2_0__r.CIF_Freight_and_Insurance__c'][0]
    client_actual_freight = so_tax_item['Client_Actual_Freight_Value__c'][0]
    client_actual_insurance = so_tax_item['Client_Actual_Insurance_Value__c'][0]
    so_has_line_items = so_tax_item['of_Line_Items__c'][0] > 0
    shipment_value = so_tax_item['Shipment_Value_USD__c'][0]
    
    if pd.isna(cpa_cif):
        so_cif_value = (shipment_value + cif_absolute_value_adjustment) * (1 + cif_adjustment_factor)
    else:
        so_cif_value = shipment_value + client_actual_freight + client_actual_insurance
        
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c'][0]
        li_proportion = li_value / shipment_value
        li_cif_value = so_cif_value * li_proportion
        return li_cif_value
    else:
        return so_cif_value

def get_fob_value(tax_structure_item,so_tax_item,li_tax_item={}):
    so_has_line_items = so_tax_item['of_Line_Items__c'][0] > 0
    so_fob_value = so_tax_item['Shipment_Value_USD__c'][0]
    
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c'][0]
        li_proportion = li_value / so_fob_value
        li_fob_value = so_fob_value * li_proportion
        return li_fob_value
    else:
        return so_fob_value

def get_previous_tax_total(so_tax_item,so_tax_totals_dict,li_tax_totals_dict,order_number,lookup_value):
    so_has_line_items = so_tax_item['of_Line_Items__c'][0] > 0
    
    if so_has_line_items:
        return li_tax_totals_dict[lookup_value][order_number]
    else:
        return so_tax_totals_dict[order_number]

In [129]:
def so_tax_totals_rollup(li_tax_totals_dict,so_tax_totals_dict):
    for tax_order in li_tax_totals_dict[list(li_tax_totals_dict.keys())[0]].keys():
        so_tax_totals_dict[tax_order] = 0
    
    for li in li_tax_totals_dict.keys():
        for tax_order in li_tax_totals_dict[li].keys():
            so_tax_totals_dict[tax_order] += li_tax_totals_dict[li][tax_order]
                    
    return so_tax_totals_dict

In [131]:
def apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number):
    tax_total = so_tax_totals_dict[tax_order_number]
    
    if not pd.isna(tax_max):
        tax_total = min(tax_total,tax_max)
   
    if not pd.isna(tax_min):
        tax_total = max(tax_total,tax_min)

    so_tax_totals_dict[tax_order_number] = tax_total
    
    return so_tax_totals_dict

# Total Taxes

In [110]:
# def main(dataset, lookup_value):
lookup_value = 'SO05'
data = data.copy()

so_records = get_SOs(data, lookup_value)
li_records = get_LIs(data, lookup_value)
clearance_country = so_records['CPA_v2_0__r.Final_Destination__c']
tax_structure = get_tax_structure(data, clearance_country[0])


In [142]:
li_tax_totals_dict = {}
so_tax_totals_dict = {}

if so_records['of_Line_Items__c'].any():
    i = 0
    for record_iter in range(0, len(li_records)):
        li_record = li_records.iloc[[record_iter]]
        li_lookup_value = li_record['Lookup Value'][0]
        li_tax_totals_dict[li_lookup_value] = {}

        for tax_iter in range(0, len(tax_structure)):
            tax_record = tax_structure.iloc[[tax_iter]]
            tax_order_number = str(tax_record['Order_Number__c'][0])
            tax_type = get_tax_type(tax_record)
            tax_conditions = get_tax_conditions(data, tax_record)

            if tax_type == 'Amount':
                tax_amount = get_tax_amount(tax_record, so_records, tax_type, li_record)
                tax_total = tax_amount
            
            if tax_type != 'Amount':
                tax_rate = get_tax_rate(tax_record, so_records, tax_type, li_record)
                taxable_value = get_taxable_value(so_tax_totals_dict, li_tax_totals_dict, tax_record, so_records, tax_type, li_record)
                tax_total = taxable_value * tax_rate
            
            li_tax_totals_dict[li_lookup_value][tax_order_number] = tax_total
        
        so_tax_totals_dict = so_tax_totals_rollup(li_tax_totals_dict,so_tax_totals_dict)

        for tax_record_iter in range(0, len(tax_structure)):
            tax_record = tax_structure.iloc[[tax_record_iter]]
            tax_order_number = str(tax_record['Order_Number__c'][0])
            tax_max = tax_record['Max__c'][0]
            tax_min = tax_record['Min__c'][0]
            so_tax_totals_dict = apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number)
    else:
        #print(so_record['Lookup Value'])
        for tax_record_iter in range(0, len(tax_structure)):
            tax_record = tax_structure.iloc[[tax_record_iter]]
            tax_order_number = str(tax_record['Order_Number__c'][0])
            tax_max = tax_record['Max__c'][0]
            tax_min = tax_record['Min__c'][0]
            tax_type = get_tax_type(tax_record)
            #print(tax_type)
            tax_conditions_dict = get_tax_conditions(data, tax_record)
            
            if tax_type == 'Amount':
                tax_amount = get_tax_amount(tax_record,so_records,tax_type)
                #print(tax_amount)
            
            if tax_type != 'Amount':
                tax_rate = get_tax_rate(tax_record,so_records,tax_type)
                taxable_value = get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_record,so_records,tax_type)
                #print(tax_rate)
                #print(taxable_value)
            #print(tax_structure_dict[tax_record_key]['Tax_Type__c'])
            
            if tax_type == 'Amount':
                tax_total = tax_amount
            else:
                tax_total = taxable_value * tax_rate
                
            #print(tax_total)
            so_tax_totals_dict[tax_order_number] = tax_total
            so_tax_totals_dict = apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number)
            #print(so_tax_totals_dict)
    
    total_tax_for_so = 0
    for order_key in so_tax_totals_dict.keys():
        total_tax_for_so += so_tax_totals_dict[order_key]
        
print(total_tax_for_so)

KeyError: 'Rate__c'

In [94]:
tax_conditions

,Id,Condition_Region__c,Tax_Structure_Per_Country__c,Condition__c,Condition_Currency__c,Condition_Value__c,Condition_Floor__c,Name,Condition_Ceiling__c,Condition_Number__c,PowerOfOne__c,Tax_Structure_Per_Country__r.Country__r.Name,Lookup Value
a6J1v000000kDSjEAM,a6J1v000000kDSjEAM,European Union,a1c1v000007cDFHAA2,Contains,US Dollar (USD),Ship From Country,NaN,TC-163,NaN,1,1,United Kingdom,A6J1V000000KDSJEAM


In [65]:
str(tax_structure.iloc[[1]]['Order_Number__c'][0])

'2'